<a href="https://colab.research.google.com/github/adong-hood/cs200/blob/main/Copy_of_Assignment10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>Assignment 10: Web Scraping Practice II</h2>

In this practice, you will extract **Business Data** from the 2017 CIA web site. Start with [Notes and Definitions](http://pluto.hood.edu/~dong/factbook-2017/docs/notesanddefs.html) page from the 2017 version.

**Your Task**
<p>Share your notebook.</p>
<h3>Step 1:</h3>
<p>There are 177 different fields in the 2017 data. Loading all of them would be a considerable amount of work, and more data than we need. Let's start with a list that is close to our original data above. <b>The list is different from our book.</b></p>
<ul>
<li>Country (name)</li>
<li>GDP - per capita (PPP)</li>
<li>GDP - Real Growth Rate</li>
</ul>


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
address = 'http://pluto.hood.edu/~dong/factbook-2017/docs/notesanddefs.html'
res = requests.get(address)
page = res.text
page_content_notesanddefs = BeautifulSoup(page, 'html.parser')
rows = page_content_notesanddefs.select("span.category") # return a list of span elements with class=category.
for row in rows:
    cells = row.select('td') # return all the td elements in one span element. There are two per tr element.
    colname = cells[0].text
    if (colname == 'Country name') | (colname == 'GDP - per capita (PPP)') | (colname == 'GDP - real growth rate'):
      links = cells[1].select('a') # return all a elements in the second td element. There is one.
      #print(links)
      if len(links) > 0:
        fpath = links[0]['href'] # href functions as the key to  ../fields/2091.html#118
        print("Field name:", colname, "\nFile Path:", fpath)


In [ ]:
address = 'http://pluto.hood.edu/~dong/factbook-2017/fields/2142.html#32'
res = requests.get(address)
page_country = res.text
page_content_country = BeautifulSoup(page_country)
dict_country_name = {}
cols = page_content_country.select("td.country")
# '''
# print(len(cols))
# print(str(cols[0]).strip())
# print(str(cols[0]).strip().split('../geos/'))
# print(str(cols[0]).strip().split('../geos/')[1][:2])
# print(cols[0].text)
# '''
for col in cols:
    code = str(col).strip().split('../geos/')[1][:2] # chained into one step. You can split into multiple steps to see how it works.
    name =  col.text
    dict_country_name[code] = name
print(len(dict_country_name))
dict_country_name

In [ ]:
address = 'http://pluto.hood.edu/~dong/factbook-2017/fields/2003.html#86'
res = requests.get(address)
page_gdp_rate = res.text
page_content_gdp_rate = BeautifulSoup(page_gdp_rate)
rows = page_content_gdp_rate.select("#fieldListing tr")
dict_gdp_rate = {}
for row in rows:
  if len(str(row).split('/geos/')) > 1:
        code = str(row).split('/geos/')[1][:2]
  cells = row.select('td')
  if len(cells) > 0:
      total_rate = cells[1].text.strip()
      dict_gdp_rate[code] = total_rate.strip().split('%')[0]
dict_gdp_rate

In [ ]:
address = 'http://pluto.hood.edu/~dong/factbook-2017/fields/2004.html#85'
res = requests.get(address)
page_gdp_percapita = res.text
page_content_gdp_percapita = BeautifulSoup(page_gdp_percapita)
rows = page_content_gdp_percapita.select("#fieldListing tr")
dict_gdp_percapita = {}
for row in rows:
  if len(str(row).split('/geos/')) > 1:
        code = str(row).split('/geos/')[1][:2]
  cells = row.select('td')
  if len(cells) > 0:
      total_rate = cells[1].text.strip()
      dict_gdp_percapita[code] = total_rate.strip().split()[0][1:].replace(',','')
dict_gdp_percapita

In [ ]:
final_df = pd.DataFrame({"Country":dict_country_name, 'GDP-per capita':dict_gdp_percapita, 'GDP-growth rate':dict_gdp_rate})
print(final_df.shape)
final_df.head()

In [ ]:
def clean_data(in_data):
  try:
    return float(in_data)
  except:
    return float('NaN')

In [ ]:
final_df['GDP-growth rate'] = final_df['GDP-growth rate'].map(clean_data)
position = final_df['GDP-growth rate'].idxmax()
final_df.loc[position]['Country']

In [ ]:
final_df['GDP-per capita'] = final_df['GDP-per capita'].map(clean_data)

In [ ]:
final_df['cca2'] = final_df.index
final_df.reset_index(drop=True, inplace=True)
final_df.head()

<h3>Step 2: </h3>
Merge the data of 2017 with those from word_countries.csv.


In [ ]:
wd = pd.read_csv('http://pluto.hood.edu/~dong/datasets/world_countries.csv')
print(wd.shape)
wd.head()

In [ ]:
final_merged_df = final_df.merge(wd, left_on='Country', right_on='Country', how = 'inner')
print(final_merged_df.shape)
final_merged_df.head()

In [ ]:
final_merged_df['gdp-diff'] = final_merged_df['GDP-per capita'] - final_merged_df['GDP']
position = final_merged_df['gdp-diff'].idxmax()
final_merged_df.loc[position]['Country']


<h3>Step 3:</h3>  

*   Find which country has the largest positive change in GDP (per capita) and which country has the largest negative change.
*   Find out which country has the largest GDP - Real Growth Rate



In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
#This is helpful for finding the exact field name.
address = 'http://pluto.hood.edu/~dong/factbook-2017/docs/notesanddefs.html'
res = requests.get(address)
page = res.text
page_content_notesanddefs = BeautifulSoup(page, 'html.parser')
spans = page_content_notesanddefs.select("span.category") # return a list of span elements with class=category.
cols_lst =[]
for aspan in spans:
    cells = aspan.select('td') # return all the td elements in one span element. There are two in the above segment.
    colname = cells[0].text
    #print(colname)
    cols_lst.append(colname)
print(len(cols_lst))
print(cols_lst)